# Data Cleaning

I downloaded this data from https://archive.ics.uci.edu/ml/datasets/wine+quality, and put the two csv sheets (one for the red wines dataset and one for the white wines dataset).

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

wines_red_data = pd.read_csv("winequality-red.csv")
wines_white_data = pd.read_csv("winequality-white.csv", delimiter=";")

The vast majority of this data was already cleaned, as I document in the data description. Each column was already an integer, so there wasn't much to clean here. I also checked for outliers or possible mislabeled data, but I believe that because this dataset is from a machine learning repository, the data came pretty clean already, with one outlier I found with a very high sugar level. However, when looking at that row, the outlier was simply a wine with a high sugar level, and I left it in as it wasn't incorrectly inputted data, but rather a data point I'd want to remain in our dataset.

In [16]:
print("red wine dataset columns")
print(wines_red_data.columns)
print("\n \ndatatypes of red wine columns")
print(wines_red_data.dtypes)
print("\n \nwhite wine dataset columns")
print(wines_white_data.columns)
print("\n \ndatatypes of white wine columns")
print(wines_white_data.dtypes)

red wine dataset columns
Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

 
datatypes of red wine columns
fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object

 
white wine dataset columns
Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

 
datatypes of white wine columns
fixed acidity           flo

We did have two separate tables, one for red and one for white. We combined the two tables, and added a column to note whether it's red or white. This will allow us to do three types of analysis and answering, one for wines overall, one for red wines, and one for white.

In [17]:
wines_red_data["wine color"] = "red"
wines_white_data["wine color"] = "white"
print("{} observations for red wine".format(len(wines_red_data)))
print("{} observations for white wine".format(len(wines_white_data)))

1599 observations for red wine
4898 observations for white wine


Because our concatenated data is severely imbalanced, this could affect our analysis when we consider the different feature importances for wine rating that depend on wine color. For example, the general important features for wine rating will be more aligned with the important features we find for white wine since the data for white wine dominate around 75% of our dataset. Therefore, we should balance the data for both types by randomly selecting 1599 samples of the 4898 white wines. 

In [18]:
wines_white_data = wines_white_data.sample(frac = .33) 

In [19]:
print("{} observations for red wine".format(len(wines_red_data)))
print("{} observations for newly sampled white wine".format(len(wines_white_data)))

1599 observations for red wine
1616 observations for newly sampled white wine


In [20]:
wines = pd.concat([wines_red_data, wines_white_data])

We reformatted column names that they'd be easier to refer to

In [21]:
new_column_names = [column.replace(' ', '_') for column in wines.columns]
new_column_names[11] = 'rating'
wines.columns = new_column_names

There's a lot of columns here, so we dropped the three acidity columns (as pH should be a good indicator for that, and we dropped the free sulfur dioxide column as well since total sulfur dioxide is present.

In [22]:
drop_columns = ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'free_sulfur_dioxide']
wines = wines.drop(drop_columns, axis='columns')

We looked at the value counts of each column, but there wasn't much to clean up here

In [23]:
for column in wines:
    print(column)
    print(wines[column].value_counts())
    print("\n")

residual_sugar
2.00     182
1.80     170
2.20     153
2.10     149
1.90     146
        ... 
7.35       1
19.45      1
19.95      1
19.40      1
12.55      1
Name: residual_sugar, Length: 264, dtype: int64


chlorides
0.042    78
0.045    72
0.046    72
0.036    68
0.080    67
         ..
0.358     1
0.422     1
0.194     1
0.133     1
0.125     1
Name: chlorides, Length: 203, dtype: int64


total_sulfur_dioxide
28.0     44
24.0     36
18.0     35
15.0     35
23.0     34
         ..
366.5     1
251.0     1
313.0     1
272.0     1
238.5     1
Name: total_sulfur_dioxide, Length: 251, dtype: int64


density
0.99720    51
0.99760    45
0.99800    40
0.99620    40
0.99680    37
           ..
0.99581     1
0.99761     1
0.99241     1
0.99235     1
0.99385     1
Name: density, Length: 860, dtype: int64


pH
3.16    98
3.30    88
3.26    81
3.22    80
3.20    80
        ..
3.77     1
2.84     1
3.64     1
2.82     1
3.85     1
Name: pH, Length: 103, dtype: int64


sulphates
0.54    118
0.50   

In [24]:
wines

,residual_sugar,chlorides,total_sulfur_dioxide,density,pH,sulphates,alcohol,rating,wine_color
0,1.9,0.076,34.0,0.9978,3.51,0.56,9.4,5,red
1,2.6,0.098,67.0,0.9968,3.20,0.68,9.8,5,red
2,2.3,0.092,54.0,0.9970,3.26,0.65,9.8,5,red
3,1.9,0.075,60.0,0.9980,3.16,0.58,9.8,6,red
4,1.9,0.076,34.0,0.9978,3.51,0.56,9.4,5,red
...,...,...,...,...,...,...,...,...,...
2172,7.8,0.042,117.0,0.9960,3.23,0.37,9.2,5,white
1847,11.5,0.049,157.0,0.9980,3.27,0.44,9.0,5,white
925,2.9,0.047,100.0,0.9936,3.28,0.43,9.8,5,white
640,1.2,0.047,127.0,0.9910,3.34,0.50,11.0,7,white


In [25]:
wines.to_csv('wine.csv',index=False)